
# Job Salary Dashboard — Colab Pipeline (IBM Granite via Replicate)

**Goal:** Use a single dataset (Data Science Job Salaries) to produce JSONs for a Next.js dashboard, and
generate **AI insights** via **IBM Granite** (Replicate) with LangChain.

**Outputs (saved into `/content/public/data/`):**
- `roles.json` — Top roles by median salary
- `countries.json` — Top countries by median salary
- `levels.json` — Salary by experience level
- `trends.json` — Median salary by year
- `insights.json` — AI summary (Granite on Replicate)


In [45]:
# === 1) Imports & setup paths ===
import os, json, math
import pandas as pd
import numpy as np
from datetime import datetime, timezone
from dateutil import parser as dateparser

# Load environment variables dari .env file
try:
    from dotenv import load_dotenv
    load_dotenv()  # Load .env file
    print("✅ Environment variables loaded from .env")
except ImportError:
    print("⚠️  python-dotenv not installed. Install with: pip install python-dotenv")
    print("   Or set REPLICATE_API_TOKEN manually in your environment")

# Path untuk environment lokal (bukan Colab) 
# Dari folder notebooks, naik 1 level ke root, lalu ke public/data
PUBLIC = '../public/data'
os.makedirs(PUBLIC, exist_ok=True)

print('Output folder:', os.path.abspath(PUBLIC))


Output folder: d:\Semester 6\job-salary-granite-starter\public\data



## 2) Set Replicate API token and initialize IBM Granite model (LangChain)
- Put your token in **Runtime → Variables** with key `api_token` (recommended), or set manually.
- Model: `ibm-granite/granite-3.3-8b-instruct` (available on Replicate).


In [46]:

# === 2) Replicate auth & model ===
from langchain_community.llms import Replicate

# Gunakan environment variable untuk API token (lebih aman)
api_token = os.getenv("REPLICATE_API_TOKEN")
if not api_token:
    print("⚠️  REPLICATE_API_TOKEN tidak ditemukan di environment variables!")
    print("   Set token dengan: export REPLICATE_API_TOKEN='your_token_here'")
    print("   Atau buat file .env dengan: REPLICATE_API_TOKEN=your_token_here")
    raise ValueError("REPLICATE_API_TOKEN environment variable required")

# Inisialisasi Granite di Replicate
llm = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    replicate_api_token=api_token
)

print("✅ Replicate model siap digunakan.")


Replicate model siap digunakan.



## 3) Load & clean
- Keep rows with year ≥ 2020 and positive salaries.
- Normalize a few text columns.


In [47]:

# Path untuk file CSV di folder notebooks
csv_path = "ds_salaries.csv"  # file ada di folder notebooks
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"File ds_salaries.csv tidak ditemukan di folder notebooks")

df = pd.read_csv(csv_path)

# Kolom yang dibutuhkan
cols_need = ["work_year","experience_level","employment_type","job_title",
             "salary_in_usd","company_location","employee_residence"]
miss = [c for c in cols_need if c not in df.columns]
if miss:
    raise ValueError(f"Kolom wajib tidak ditemukan: {miss}")

df = df.copy()
df["work_year"] = pd.to_numeric(df["work_year"], errors="coerce")
df = df[(df["work_year"] >= 2020) & (df["salary_in_usd"] > 0)]

for c in ["job_title","company_location","employee_residence","experience_level","employment_type"]:
    df[c] = df[c].astype(str).str.strip()

df["experience_level"] = df["experience_level"].str.upper()  # EN, MI, SE, EX, dll

print("Jumlah baris setelah cleaning:", len(df))
df.head()


Jumlah baris setelah cleaning: 607


,Unnamed: 0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L



## 5) Aggregations → roles, countries, levels, trends

In [48]:

def pct(x, p):
    try:
        return float(np.nanpercentile(x, p)) if len(x) else None
    except Exception:
        return None

# Filter parameter untuk menghilangkan outliers dengan sample kecil
MIN_ROLES_SAMPLE = 3   # minimal 3 data points untuk roles
MIN_COUNTRIES_SAMPLE = 5  # minimal 5 data points untuk countries

# a) Roles (top 20 by median) - Filter out outliers dengan sample kecil
roles = (df.groupby("job_title")["salary_in_usd"]
           .agg(p25=lambda x: pct(x,25), p50=lambda x: pct(x,50), p75=lambda x: pct(x,75), n="count")
           .reset_index()
           .sort_values("p50", ascending=False))
roles_top = roles[roles["n"] >= MIN_ROLES_SAMPLE].head(20)  # Filter n≥3, ambil top 20
roles_json = {
    "meta": {
        "generated_at": datetime.now(timezone.utc).isoformat(), 
        "k": len(roles_top),
        "currency": "USD",
        "sort_by": "p50_desc",
        "min_sample_size": MIN_ROLES_SAMPLE
    },
    "items": [
        {"role": r["job_title"], "p25": r["p25"], "p50": r["p50"], "p75": r["p75"], "n": int(r["n"])}
        for _, r in roles_top.iterrows()
    ]
}

# b) Countries (top 25 by median) - Filter out outliers dengan sample kecil
countries = (df.groupby("company_location")["salary_in_usd"]
               .agg(p25=lambda x: pct(x,25), p50=lambda x: pct(x,50), p75=lambda x: pct(x,75), n="count")
               .reset_index()
               .sort_values("p50", ascending=False))
countries_top = countries[countries["n"] >= MIN_COUNTRIES_SAMPLE].head(25)  # Filter n≥5, ambil top 25
countries_json = {
    "meta": {
        "generated_at": datetime.now(timezone.utc).isoformat(), 
        "k": len(countries_top),
        "currency": "USD",
        "sort_by": "p50_desc",
        "min_sample_size": MIN_COUNTRIES_SAMPLE
    },
    "items": [
        {"country": r["company_location"], "p25": r["p25"], "p50": r["p50"], "p75": r["p75"], "n": int(r["n"])}
        for _, r in countries_top.iterrows()
    ]
}

# c) Levels
levels = (df.groupby("experience_level")["salary_in_usd"]
            .agg(p25=lambda x: pct(x,25), p50=lambda x: pct(x,50), p75=lambda x: pct(x,75), n="count")
            .reset_index()
            .sort_values("p50", ascending=False))
levels_json = {
    "meta": {
        "generated_at": datetime.now(timezone.utc).isoformat(),
        "currency": "USD",
        "sort_by": "p50_desc"
    },
    "items": [
        {"level": r["experience_level"], "p25": r["p25"], "p50": r["p50"], "p75": r["p75"], "n": int(r["n"])}
        for _, r in levels.iterrows()
    ]
}

# d) Trends (median per year)
trends = (df.groupby("work_year")["salary_in_usd"]
            .agg(p50=lambda x: pct(x,50), n="count")
            .reset_index()
            .sort_values("work_year"))
# Get latest year from trends
latest_year = max([s["year"] for s in trends_json["series"]]) if trends_json["series"] else None

trends_json = {
    "meta": {
        "generated_at": datetime.now(timezone.utc).isoformat(),
        "currency": "USD",
        "latest_year": latest_year
    },
    "series": [
        {"year": int(r["work_year"]), "p50": r["p50"], "n": int(r["n"])}
        for _, r in trends.iterrows()
    ]
}

# Save all
with open(f"{PUBLIC}/roles.json","w") as f: json.dump(roles_json, f, indent=2)
with open(f"{PUBLIC}/countries.json","w") as f: json.dump(countries_json, f, indent=2)
with open(f"{PUBLIC}/levels.json","w") as f: json.dump(levels_json, f, indent=2)
with open(f"{PUBLIC}/trends.json","w") as f: json.dump(trends_json, f, indent=2)

print("Saved roles.json, countries.json, levels.json, trends.json to:", PUBLIC)


Saved roles.json, countries.json, levels.json, trends.json to: ../public/data



## 6) AI Insight (Granite via Replicate)
- Builds a compact context from aggregates.
- Prompts Granite to return **5 bullets + 1 recommendation** in Indonesian.


In [49]:

# Load data untuk insights yang tervalidasi
with open(f"{PUBLIC}/roles.json") as f: roles = json.load(f)
with open(f"{PUBLIC}/countries.json") as f: countries = json.load(f)
with open(f"{PUBLIC}/levels.json") as f: levels = json.load(f)
with open(f"{PUBLIC}/trends.json") as f: trends = json.load(f)

top_role = roles["items"][0] if roles["items"] else None
top_country = countries["items"][0] if countries["items"] else None

# Growth yang valid dari trends (overall)
y = {s["year"]: s["p50"] for s in trends["series"]}
g_21_22 = round((y[2022]-y[2021]) / y[2021] * 100, 1) if 2021 in y and 2022 in y else None
g_20_22 = round((y[2022]-y[2020]) / y[2020] * 100, 1) if 2020 in y and 2022 in y else None

# Rasio SE/EN
get = lambda L,k: next((x["p50"] for x in L["items"] if x["level"]==k), None)
se, en = get(levels,"SE"), get(levels,"EN")
ratio_se_en = round(se/en, 2) if se and en else None

# FACTS yang tervalidasi
FACTS = {
    "top_role": top_role,                 # role global (sudah n≥3)
    "top_country": top_country,           # country (sudah n≥5)
    "levels": levels["items"],            # EX/SE/MI/EN
    "trends": trends["series"],           # 2020..2022
    "growth_pct": {"2021_to_2022": g_21_22, "2020_to_2022": g_20_22},
    "ratio_SE_to_EN": ratio_se_en,
    "notes": {"currency":"USD", "filters":f"roles≥{roles['meta']['min_sample_size']}, countries≥{countries['meta']['min_sample_size']}"}
}

# Prompt yang ketat untuk Granite
prompt = f"""
Anda analis data yang ketat. Gunakan HANYA angka dari FACTS berikut.
JANGAN menghitung ulang; cukup parafrase dan formatkan angka (pakai $ & pemisah ribuan).
Jika sampel kecil (n<5), sebutkan 'sampel kecil'.

Keluarkan **JSON** persis dengan schema:
{{"bullets": ["...", "...", "...", "...", "..."], "recommendation": "..."}}

FACTS:
{json_lib.dumps(FACTS, ensure_ascii=False)}
"""

# Generate insights dengan Granite
llm = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    model_kwargs={"temperature":0.1, "max_new_tokens":400},
    replicate_api_token=os.environ["REPLICATE_API_TOKEN"]
)

raw = llm.invoke(prompt).strip()
try:
    out = json.loads(raw)
except Exception:
    out = {"bullets":[raw], "recommendation": ""}

# Buat insights.json dengan format yang konsisten
insights_json = {
    "meta": {
        "generated_at": datetime.now(timezone.utc).isoformat(),
        "model": "ibm-granite/granite-3.3-8b-instruct",
        "format": "json",
        "currency": "USD"
    },
    **out  # bullets & recommendation dari Granite
}

with open(f"{PUBLIC}/insights.json","w") as f: json.dump(insights_json, f, indent=2)

print("Saved insights.json to:", PUBLIC)
print(f"\n=== Preview ===")
print(f"Bullets: {len(insights_json['bullets'])} items")
for i, bullet in enumerate(insights_json['bullets'][:3], 1):
    print(f"{i}. {bullet[:100]}...")
print(f"Recommendation: {insights_json['recommendation'][:150]}...")


Saved insights.json to: ../public/data

=== Preview ===
Bullets: 1 items
1. ```json
{
  "bullets": [
    "The median salary for Head of Data role is $200,000 with 25th percenti...
Recommendation: ...


In [50]:
# Fix insights.json structure (remove nested JSON in string)
import json

path = f"{PUBLIC}/insights.json"  # gunakan PUBLIC dari cell sebelumnya
with open(path) as f:
    data = json.load(f)

# Jika bullets = [ "<JSON string>" ], parse dan rapikan
if isinstance(data.get("bullets"), list) and len(data["bullets"]) == 1 and isinstance(data["bullets"][0], str):
    inner = data["bullets"][0]
    # ambil substring JSON pertama yang valid
    try:
        start = inner.index("{")
        end = inner.rindex("}")
        repaired = json.loads(inner[start:end+1])
    except Exception:
        repaired = {}

    bullets = repaired.get("bullets") or []
    rec = repaired.get("recommendation") or data.get("recommendation") or ""

    fixed = {
        "meta": {
            **data.get("meta", {}), 
            "format": "json", 
            "source": "granite_repaired",
            "currency": "USD"  # tambahkan currency untuk konsistensi
        },
        "bullets": [str(b).strip() for b in bullets][:5],  # maksimal 5 bullet
        "recommendation": str(rec).strip()
    }

    with open(path, "w") as f:
        json.dump(fixed, f, indent=2)
    print("Repaired insights.json ✅")
    print("\nPreview:")
    print(json.dumps(fixed, indent=2)[:200] + "...")  # show first ~200 chars
else:
    print("insights.json already looks fine.")


Repaired insights.json ✅

Preview:
{
  "meta": {
    "generated_at": "2025-09-07T16:18:54.550706+00:00",
    "model": "ibm-granite/granite-3.3-8b-instruct",
    "format": "json",
    "currency": "USD",
    "source": "granite_repaired"
...


In [51]:
# === REPAIR insights.json ===
# Fix JSON string yang nested di dalam bullets array
import json, os

path = f"{PUBLIC}/insights.json"
with open(path) as f:
    data = json.load(f)

# Jika bullets = [ "<JSON string>" ], parse dan rapikan
if isinstance(data.get("bullets"), list) and len(data["bullets"]) == 1 and isinstance(data["bullets"][0], str):
    inner = data["bullets"][0]
    # ambil substring JSON pertama yang valid
    try:
        start = inner.index("{")
        end = inner.rindex("}")
        repaired = json.loads(inner[start:end+1])
    except Exception:
        repaired = {}

    bullets = repaired.get("bullets") or []
    rec = repaired.get("recommendation") or data.get("recommendation") or ""

    fixed = {
        "meta": {**data.get("meta", {}), "format": "json", "source": "granite_repaired"},
        "bullets": [str(b).strip() for b in bullets][:5],  # maksimal 5 bullet
        "recommendation": str(rec).strip()
    }

    with open(path, "w") as f:
        json.dump(fixed, f, indent=2)
    print("Repaired insights.json ✅")
    print(f"Bullets count: {len(fixed['bullets'])}")
    print(f"First bullet: {fixed['bullets'][0][:100]}...")
else:
    print("insights.json already looks fine.")


insights.json already looks fine.
